### EDA IOWA Dataset 03 - Fixing Missing Values in Counties

**Status:** PUBLIC Distribution <br>

**Author:** Jaume Manero IE<br>
**Date created:** 2021/02/1<br>
**Last modified:** 2024/03/20 <br>
**Description:** Fixing missing values

The original dataset has many missing values in the county. This is an example on how to fix it

In [1]:
import pandas as pd
import numpy as np
import time
import uszipcode
from uszipcode import SearchEngine
%matplotlib inline

In [2]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return hours, minutes, seconds

In [3]:
start_time = time.time()   # it takes some time we have almost 30 million rows

file = 'Iowa_Liquor_Sales_DEC2023.csv'
df = pd.read_csv(file, header=0)

C:\Users\USER\AppData\Local\Temp\ipykernel_17516\2851729181.py:4: DtypeWarning: Columns (6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, header=0)


In [4]:
df.dtypes

Invoice/Item Number       object
Date                      object
Store Number               int64
Store Name                object
Address                   object
City                      object
Zip Code                  object
Store Location            object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number            float64
Vendor Name               object
Item Number               object
Item Description          object
Pack                       int64
Bottle Volume (ml)         int64
State Bottle Cost        float64
State Bottle Retail      float64
Bottles Sold               int64
Sale (Dollars)           float64
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

In [5]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

In [6]:
# how many missing values in County?
sum(pd.isnull(df['County']))

159892

In [7]:
# and in County Number?
sum(pd.isnull(df['County Number']))

3801878

In [8]:
# How many empty zip codes?
sum(pd.isnull(df['Zip Code']))

83156

In [9]:
# in Both?

df.isna().pivot_table(index='County', columns='County Number', aggfunc='size').stack()

County  County Number
False   False            24132843.0
        True              3641986.0
True    True               159892.0
dtype: float64

In [10]:
# Conclusion Empty County has Empty County Number
# Do they have Zip Code?
df.isna().pivot_table(index='County', columns='Zip Code', aggfunc='size').stack()

County  Zip Code
False   False       27774807
        True              22
True    False          76758
        True           83134
dtype: int64

In [11]:
# We have 83134 empty zip codes. I think we can delete them
# Lets create County from ZIP CODE
# We can use a package uszipcode


search = SearchEngine()
zipcode = search.by_zipcode("57001")
zipcode.county

# we create a small function that returns county from ZIPCODE
# Careful We create County in uppercase without the word county
# We found some zip codes that are not in the package (we don't know why) we filter them one by one. 
# Three of them are possibly legacy
def reverseZIP(zipcode):
    if pd.isna(zipcode):
        return zipcode
    try:
       zipcode = int(zipcode)
    except:
       return np.nan
    z = search.by_zipcode(zipcode)

    if z is None:
        if zipcode == 50015:
            return 'POLK'
        elif zipcode == 50300:
           return 'DES MOINES'
        elif zipcode == 52084:
           return 'DES MOINES'   # careful confirm it
        elif zipcode == 52087:
           return 'DUBUQUE'
        elif zipcode == 52303:
           return 'LINN'
        elif zipcode == 52671:
           return 'DES MOINES'   # not confirmed
        elif zipcode == 57222:
           return 'DES MOINES'   # not confirmed
    else:
        county = z.county
        county = county.replace(' County','')
        return county.upper()

print (reverseZIP(50015))

Download C:\Users\USER\.uszipcode\simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!
POLK


In [12]:
print (reverseZIP(57101))

MINNEHAHA


In [13]:
# now we replace the county name in the WHOLE dataset from its ZIP
# we do that because in this way we assure that some changes of County boundaries are avoided 
# and then the whole dataset is consistent

In [ ]:
df['New County'] = df['Zip Code'].apply(reverseZIP)

In [ ]:
# time keeping
end_time_part1= time.time()
elapsed_time_p1 = end_time_part1 - start_time
print(f"Total time of Part 1 (ReverseZIP) : {elapsed_time_p1} seconds")

hours, minutes, seconds = seconds_to_hms(elapsed_time_p1)

print(f"Total time of Part 1 (ReverseZIP):{hours} hours, {minutes} minutes, {seconds} seconds")

In [ ]:
# Now in the column New County we have the correct county and we have very few missing values
# You can proceed to delete the rows that are with Nan/Nulls as we don't have information (or is not worth to find it)
print('Number of nulls now in County', sum(pd.isnull(df['New County'])))
# The instruction for deletion is:
df = df.dropna(subset=['New County'])
# number of rows in the final dataframe
num_rows = len(df)
print(f"Number of rows: {num_rows}")

In [ ]:
lst = df['New County'].tolist()
print(np.unique(lst).tolist())
df['New County'].value_counts()

In [ ]:
# Now we will create a new row with the FIPS code we load a .csv I created with FIPS-NAME
# However there are a couple of format issues. In the table, Counties are in Capital+lowercase, but in new county are in uppercase
# FIPS is a sequential number but we want the structure with the IOWA prefix (19) (instead of 1 we want 19001)
# Easy, we just create another function that does the transformation
# Careful, There are duplicates as there are IOWA and IOWA COUNTY in the file

In [ ]:
file = 'Iowa_FIPS.csv'
fipsdf = pd.read_csv(file,names=['FIPS','County'], header=0)

In [ ]:
fipsdf

In [ ]:
# FIPS format is 19(IOWA)+County number(3 digits)
#
# This is not an efficient function. We could speed it up by loading the table in memory or using a dictionary
# Wanna try to modify it?
# 
def FIPS_from_county(name):
    name = name.lower()
    name = name.title() # Capitalizes all words in a string
    try:
       x = fipsdf[fipsdf.County == name].FIPS.iloc[0]
    except:
       print(name)
       stop
    x = str(x)
    x = x.zfill(5)
    x = "19" + x[2:]
    return x

FIPS_from_county('Iowa')

In [ ]:
df['New FIPS'] = df['New County'].apply(FIPS_from_county)

In [ ]:
len(df)

In [ ]:
df.to_csv('Iowa_Liquor_Sales_NOV23(cleaned).csv')

In [ ]:
# time keeping
end_time= time.time()
elapsed_time_part2 = end_time - end_time_part1
elapsed_time = end_time - start_time
print(f"Total time for part 2 FIPS from county num : {elapsed_time_part2} seconds")
hours, minutes, seconds = seconds_to_hms(elapsed_time_part2)
print(f"Total time for part 2 :{hours} hours, {minutes} minutes, {seconds} seconds")


def seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return hours, minutes, seconds

hours, minutes, seconds = seconds_to_hms(elapsed_time)

print(f"Total time of this program : {elapsed_time} seconds")
print(f"Total time of this program:{hours} hours, {minutes} minutes, {seconds} seconds")

In [ ]:
# There is a ZIP Code that has some wrong data, the 999, because Counties in ElPaso or Minessota do not make sense. 
# This happens often in large datasets where some elements of data are clearly wrong.
# My recommendation would be to delete those rows
# we have over 27 million rows with County on it

In [ ]:
import session_info
session_info.show(html=False)